<a href="https://colab.research.google.com/github/moiraone/hub_zone_dod/blob/main/Cleaning_compiling_investment_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
pip install geopandas

**Start by reading in sample file and drafting a program to compile and clean data**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import geopandas as gpd

**California sample records 2000-2020**


In [ ]:
ca = pd.read_csv(r"/content/drive/MyDrive/Master_defense_investment_data/California_contracts.csv")

In [ ]:
ca_zip = gpd.read_file(r'/content/drive/MyDrive/Spatiotemp/final project/ca shapefiles/ca_zip.shp')

In [ ]:
ca['Date Signed'].astype(str)

0         Feb 5, 2001
1         Mar 7, 2002
2        Jun 30, 2000
3         Apr 3, 2002
4        Jun 25, 2002
             ...     
29995     Apr 7, 2020
29996    Sep 22, 2020
29997    Mar 15, 2021
29998     Jun 4, 2015
29999    Jul 11, 2016
Name: Date Signed, Length: 30000, dtype: object

In [ ]:
year = []
for i in ca['Date Signed']:
  year.append(i[7:])

for i in year:
  str(i)
  i.strip()

In [ ]:
%%capture
year

In [ ]:
ca['Year'] = year

In [ ]:
ca['Year'] = ca['Year'].str.strip()

In [ ]:
ca['ZIP'] = ca['Entity ZIP Code'].str[:5]

Join investment data to zip shapefile.

In [ ]:
ca_zip['ZIP'] = ca_zip['zcta']

In [ ]:
ca_spatial = ca_zip.merge(ca, on='ZIP')

In [ ]:
#list(ca_spatial)
#ca_spatial.drop(['age_under5','age_5to9','age_10to14','age_15to19','age_20to24','age_25to29','age_30to34','age_35to39','age_40to44','age_45to49','age_50to54','age_55to59','age_60to64','age_65to69','age_70to74','age_75to79','age_80to84','age_85_ove','age16_over','age18_over','age21_over','age62_over','age65_over','male','female'], axis=0, inplace=False)

Condense records by summing investment per zip.

In [ ]:
ca_spatial['Action Obligation ($)'] = ca_spatial['Action Obligation ($)'].str.replace(',', '').str.replace('$', '')

In [ ]:
ca_spatial['Action Obligation ($)']= ca_spatial['Action Obligation ($)'].astype(float)

In [ ]:
ca_spatial['Action Obligation ($)'].astype(int)

0          3769
1         76000
2        157000
3             0
4             0
          ...  
29787      2750
29788     20000
29789      4260
29790     25785
29791     71046
Name: Action Obligation ($), Length: 29792, dtype: int64

In [ ]:
ca_sum = ca_spatial.groupby(['Year', 'ZIP'],as_index=False)['Action Obligation ($)'].sum()

In [ ]:
pd.DataFrame(ca_sum)

,Year,ZIP,Action Obligation ($)
0,1980,92122,138000.00
1,1981,92122,1420000.00
2,1982,92122,1324000.00
3,1983,92122,2282000.00
4,1983,92704,1799000.00
...,...,...,...
874,2021,94043,325491.95
875,2021,94520,0.00
876,2021,95642,397314.75
877,2021,95742,6285539.20


In [ ]:
ca_spatial = ca_zip.merge(ca_sum, on='ZIP')

In [ ]:
ca_spatial.shape

(879, 89)